# ECG Split 1D-CNN Client Side

This code is the server part of ECG split 1D-CNN model for **single** client and a server.

## Import required packages

In [1]:
import os
import struct
import socket
import pickle
import time

import h5py
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam, SGD

## Define ECG dataset class

In [2]:
batch_size = 32

import torchvision
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

root_path = '/path/to/cifar10/'  # Replace with actual path to CIFAR10 data
train_name = 'train.h5'  # Adjust filenames if necessary
test_name = 'test.h5'

batch_size = 32

# Download CIFAR10 dataset if not already present
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = CIFAR10(root=root_path, train=True, download=True, transform=transform)
test_dataset = CIFAR10(root=root_path, train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Your training and testing logic can go here, using train_loader and test_loader
# Remember to replace ECG-specific references with appropriate methods for CIFAR10 data

print("Loaded CIFAR10 datasets with batch size", batch_size)


Files already downloaded and verified
Files already downloaded and verified
Loaded CIFAR10 datasets with batch size 32


In [3]:
class EcgClient(nn.Module):
    def __init__(self):
        super(EcgClient, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 7, padding=3)
        self.relu1 = nn.LeakyReLU()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(16, 16, 5, padding=2)
        self.relu2 = nn.LeakyReLU()
        self.pool2 = nn.MaxPool2d(2)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(-1, 8 * 8 * 16)
        return x

In [4]:
device = torch.device('cuda:0')
torch.cuda.get_device_name(0)
assert('cuda' in device.type)

### Set random seed

In [5]:
seed = 0
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

### Assign intial weight as same as non-split model

In [6]:
ecg_client = EcgClient()
ecg_client.to(device)

checkpoint = torch.load("init_weight.pth")
ecg_client.conv1.weight.data = checkpoint["conv1.weight"]
ecg_client.conv1.bias.data = checkpoint["conv1.bias"]
ecg_client.conv2.weight.data = checkpoint["conv2.weight"]
ecg_client.conv2.bias.data = checkpoint["conv2.bias"]

### Set other hyperparameters in the model
Hyperparameters here should be same with the server side.

In [7]:
epoch = 400
criterion = nn.CrossEntropyLoss()
lr = 0.001
optimizer = Adam(ecg_client.parameters(), lr=lr)

## Socket initialization

### Required socket functions

In [8]:
def send_msg(sock, msg):
    # prefix each message with a 4-byte length in network byte order
    msg = struct.pack('>I', len(msg)) + msg
    sock.sendall(msg)

def recv_msg(sock):
    # read message length and unpack it into an integer
    raw_msglen = recvall(sock, 4)
    if not raw_msglen:
        return None
    msglen = struct.unpack('>I', raw_msglen)[0]
    # read the message data
    return recvall(sock, msglen)

def recvall(sock, n):
    # helper function to receive n bytes or return None if EOF is hit
    data = b''
    while len(data) < n:
        packet = sock.recv(n - len(data))
        if not packet:
            return None
        data += packet
    return data

### Set host address and port number

In [9]:
host = 'localhost'
port = 10080
max_recv = 4096

### Open the client socket

In [10]:
s = socket.socket()
s.connect((host, port))

## Real training process

In [11]:
for e in range(epoch):
    print("Epoch {} - ".format(e+1), end='')
    
    for _, batch in enumerate(train_loader):
        x, label = batch
        x, label = x.to(device), label.to(device)
        optimizer.zero_grad()
        ecg_client = ecg_client.to(device)
        output = ecg_client(x)
        client_output = output.clone().detach().requires_grad_(True)
        msg = {
            'client_output': client_output,
            'label': label
        }
        msg = pickle.dumps(msg)
        send_msg(s, msg)
        msg = recv_msg(s)
        client_grad = pickle.loads(msg)
        output.backward(client_grad)
        optimizer.step()
            
    with torch.no_grad():  # calculate test accuracy
        for _, batch in enumerate(test_loader):
            x, label = batch
            x, label = x.to(device), label.to(device)
            client_output = ecg_client(x)
            msg = {
                'client_output': client_output,
                'label': label
            }
            msg = pickle.dumps(msg)
            send_msg(s, msg)
    
    msg = recv_msg(s)
    train_test_status = pickle.loads(msg)
    print(train_test_status)

Epoch 1 - 

In [ ]:
print('Finished Training!')
print('Result is on the server side.')

Finished Training!
Result is on the server side.
